In [ ]:
def even_range(start, end):
    current = start
    while current < end:
        yield current
        print('step')
        current += 2

for number in even_range(0, 10):
    print(number)

0
step
2
step
4
step
6
step
8
step


In [ ]:
ranger = even_range(0, 4)
next(ranger)

0

In [ ]:
next(ranger)

step


2

In [ ]:
next(ranger)

step


StopIteration: ignored

Мы получили ошибку, т.к. у генератора больше нет значений, которые он может выдать.

Можем проверить, что функция действительно прерывается каждый раз после выполнения yield:

In [ ]:
def list_generator(list_obj):
    for item in list_obj:
        yield item
        print('After yielding {}'.format(item))
        
generator = list_generator([1, 2])
next(generator)

1

In [ ]:
next(generator)

After yielding 1


2

In [ ]:
next(generator)

After yielding 2


StopIteration: ignored

Когда применяются генераторы? Они нужны, например, тогда, когда мы хотим итерироваться по большому количеству значений, но не хотим загружать ими память. Именно
поэтому стандартная функция range() реализована как генератор (впрочем, так было не
всегда).
Приведём классический пример про числа Фибоначчи

In [33]:
def fibonacci(number):
    a = b = 1
    for _ in range(number):
        yield b
        a, b = b, a + b

for num in fibonacci(10):
    print(num)

1
2
3
5
8
13
21
34
55
89


С таким генератором нам не нужно помнить много чисел Фибоначчи, которые быстро
растут --- достаточно помнить два последних числа.

Еще одна важная особенность генераторов --- это возможность передавать генератору
какие-то значения. Эта особенность активно используется в асинхронном программировании, о котором будет речь позднее. Пока определим генератор accumulator, который
хранит общее количество данных и в бесконечном цикле получает с помощью оператора
yield значение. На первой итерации генератор возвращает начально значение total.
После этого мы можем послать данные в генератор с помощью метода генератора send.
Поскольку генератор остановил исполнение в некоторой точке, мы можем послать в эту
точку значение, которое запишется в value. Далее, если value не было передано, генератор выходит из цикла, иначе прибавляем его к total.

In [19]:
def accumulator():
    total = 0
    while True:
        value = yield total
        print('Got: {}'.format(value))
        if not value: break
        total += value

generator = accumulator()
next(generator)

0

In [20]:
print('Accumulated: {}'.format(generator.send(1)))

Got: 1
Accumulated: 1


In [21]:
print('Accumulated: {}'.format(generator.send(1)))

Got: 1
Accumulated: 2


In [22]:
print('Accumulated: {}'.format(generator.send(7)))

Got: 7
Accumulated: 9


In [23]:
print('Accumulated: {}'.format(generator.send(1)))

Got: 1
Accumulated: 10


In [24]:
next(generator)

Got: None


StopIteration: ignored

Как вы уже поняли, yield – двухсторонний оператор. Сначала генераторная функция передает значение вызывающей стороне (yield something). Затем останавливается и ждет, пока вызывающая сторона не передаст ей что-нибудь в ответ (generator.send(something)), чтобы она могла сохранить это значение (something = yield) и продолжить работу до следующего оператора yield.

yield является двухсторонним оператором всегда, даже если вы не передаете значение генератору через send, а просто пытаетесь продолжить работу генератора через next:

In [25]:
def hello():
    value = yield 'Hello'
    print('value = {}'.format(value))

gen = hello()

In [26]:
next(gen)

'Hello'

In [27]:
next(gen)

value = None


StopIteration: ignored

Аналогично и в ситуации, когда yield используется только для получения данных. В таком случае, yield передает None вызывающей стороне:

In [28]:
def simple_coroutine():
    value = yield
    print(value)

In [29]:
coro = simple_coroutine()

In [30]:
from_coro = next(coro)

In [31]:
print(from_coro)

None


In [32]:
coro.send(42)

42


StopIteration: ignored